In [1]:
import pandas as pd
import numpy
from __future__ import division

In [2]:
df1_WS2T = pd.read_csv("/Users/asteriag/Desktop/DATASCIENCEPROJECT_WORKFLOW/TimeStamp_Engineering/engineering_OF_segment1_workflowS2_has_duration.csv",engine='python')

In [3]:
df1_WS2T.head(n=1)

,CASE_ID,WRKFLW_TSK_MAP_GROUPED,WRKFLW_TSK_MAP_DETAILED,PATH_CRDATETIME,ITEM_CRUSER,WRKFLW_ORG_TYPE_CRUSER,STATUS_ORDER,LATEST_STATUS,WRKFLW_PROCESS,QUERY_PART,...,SPK_CAT_TYPE_CD,BROAD_COURSE_LEVEL,DEGREETYPE,LIABILITY_CATEGORY_CD_RECODE,ORIG_LIABILITY_CATEGORY_CD_RECODE,OFFER_RELEASED_FG,OFFER_OUTCOME_CD,OFFER_CONDITIONS,time_used_str,time_used_second
0,1126888|499828|767443,Send for Assessment,Send for Assessment,17/12/2019 14:06:44,PRESTONZ,Academic Services,9,N,1. App to Assessment,Part 1 - Application ID Workflow,...,MCW,HDCW,Full Degree,OF,OF,Y,NaN,Y,21 days 21 hours 36 minutes 6 seconds,1892166.0


In [4]:
numOfWorkflowProcesses = df1_WS2T.groupby('CASE_ID').CASE_ID.count()

count_dic=dict()
for i,v in numOfWorkflowProcesses.items():
    if count_dic.get(v) is None:
        count_dic[v]=[]
        count_dic[v].append(i)
    else:
        count_dic[v].append(i)
#count_dic
count_dic = dict(sorted(count_dic.items()))

In [5]:
len_list=[]
for key in count_dic.keys():
    len_list.append(len(count_dic[key]))

In [6]:
def strfdelta(tdelta, fmt):
    d=dict()
    d["days"],rem1 = divmod(tdelta,60*60*24)
    d["hours"], rem2 = divmod(rem1, 3600)
    d["minutes"], d["seconds"] = divmod(rem2, 60)
    return fmt.format(**d)

In [7]:
list_avgtime_per_key =[]
for key in count_dic.keys():
    id_list = count_dic[key]
    list_sumtime_person = 0
    for id in id_list:
        df = df1_WS2T[df1_WS2T['CASE_ID']== id]
        tsum = df['time_used_second'].sum()
        list_sumtime_person+=tsum
    list_avgtime_per_key.append(int((list_sumtime_person/len(count_dic[key]))))

list_avgtime_per_key_str = []
for i in list_avgtime_per_key:
    list_avgtime_per_key_str.append(strfdelta(i,"{days} days {hours} hours {minutes} minutes {seconds} seconds"))

In [8]:
list_mediantime_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_sumtime_per_person = []
    for id in id_list:
        df = df1_WS2T[df1_WS2T['CASE_ID']== id]
        tsum = df['time_used_second'].sum()
        list_sumtime_per_person.append(tsum)
    list_mediantime_per_key.append(numpy.median(list_sumtime_per_person))

list_mediantime_per_key_str = []
for i in list_mediantime_per_key:
    list_mediantime_per_key_str.append(strfdelta(i,"{days} days {hours} hours {minutes} minutes {seconds} seconds"))

In [9]:
list_percent_reach_end = []
for key in count_dic.keys():
    id_list = count_dic[key]
    count_df_end_step = 0
    count_df_id = 0
    for id in id_list:
        df_id = df1_WS2T[df1_WS2T['CASE_ID']== id]
        df_end_step = df_id[df_id['WRKFLW_TSK_MAP_GROUPED'].isin(['Assessment Outcome - Successful','Assessment Outcome - Not Successful']) ]
        count_df_id += df_id['CASE_ID'].nunique()
        count_df_end_step += df_end_step['CASE_ID'].nunique()
    list_percent_reach_end.append(str(round((count_df_end_step/count_df_id)*100,2))+ "%")

In [10]:
list_step_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_step = set()
    for id in id_list:
        df = df1_WS2T[df1_WS2T['CASE_ID']== id]
        for index, row in df.iterrows():
            list_step.add(row['WRKFLW_TSK_MAP_GROUPED'])
    list_step_per_key.append(list_step)

In [11]:
from collections import Counter
import collections, functools, operator
list_step_sum_time_per_key = []
list_step_count_per_key = []
list_step_ave_time_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_step = []
    step_time_list = []
    for id in id_list:
        df = df1_WS2T[df1_WS2T['CASE_ID']== id]
        step_time = dict(zip(df.WRKFLW_TSK_MAP_GROUPED, df.time_used_second))
        step_time_list.append(step_time)
        for index, row in df.iterrows():
            list_step.append(row['WRKFLW_TSK_MAP_GROUPED'])

    step_sum_time = dict(functools.reduce(operator.add,
         map(collections.Counter, step_time_list)))

    dict_list_step=dict(Counter(list_step))

    step_ave_time = {key: strfdelta(step_sum_time[key] // dict_list_step.get(key, 0),"{days} days {hours} hours {minutes} minutes {seconds} seconds")
                        for key in step_sum_time.keys()}

    sort_step_sum_time = sorted(step_sum_time.items(),key = lambda x: x[1], reverse = True)
    list_step_sum_time_per_key.append(sort_step_sum_time)

    sort_dict_list_step = sorted(dict_list_step.items(),key = lambda x: x[1], reverse = True)
    list_step_count_per_key.append(sort_dict_list_step)

    sort_step_ave_time = sorted(step_ave_time.items(),key = lambda x: x[1], reverse = True)
    list_step_ave_time_per_key.append(sort_step_ave_time)

In [12]:
s1_t2=pd.DataFrame({'Number of Workflow processes in this Workflow Segment':count_dic.keys(),'Number of cases':len_list, 'average time for workflow segment':list_avgtime_per_key_str,'median time for workflow segment':list_mediantime_per_key_str,'percentage of cases reaching to end of workflow segment':list_percent_reach_end,'step':list_step_per_key})
s1_t2['Num of Occurrences per step(descending)'] = list_step_count_per_key
s1_t2['Average time spends per step(descending)'] = list_step_ave_time_per_key
s1_t2.head(n=10)


,Number of Workflow processes in this Workflow Segment,Number of cases,average time for workflow segment,median time for workflow segment,percentage of cases reaching to end of workflow segment,step,Num of Occurrences per step(descending),Average time spends per step(descending)
0,1,835,0 days 0 hours 0 minutes 0 seconds,0.0 days 0.0 hours 0.0 minutes 0.0 seconds,0.0%,{Send for Assessment},"[(Send for Assessment, 835)]",[]
1,2,855,26 days 15 hours 0 minutes 11 seconds,15.0 days 18.0 hours 25.0 minutes 44.0 seconds,62.69%,"{Put Assessment On-Hold, AR - Satisfied - fina...","[(Send for Assessment, 856), (Assessment Outco...","[(Send for Assessment, 26.0 days 14.0 hours 15..."
2,3,2822,30 days 12 hours 9 minutes 52 seconds,18.0 days 22.0 hours 11.0 minutes 20.0 seconds,91.11%,"{Put Assessment On-Hold, AR - Satisfied - fina...","[(Send for Assessment, 2824), (Assessment Outc...","[(AR - Satisfied - specialisation, 94.0 days 2..."
3,4,2184,40 days 0 hours 39 minutes 0 seconds,27.0 days 21.0 hours 59.0 minutes 27.0 seconds,92.12%,"{Put Assessment On-Hold, OC - Created - applic...","[(Send for Assessment, 2194), (Assessment Deci...","[(AR - Created - further information, 8.0 days..."
4,5,1577,45 days 17 hours 36 minutes 8 seconds,30.0 days 1.0 hours 9.0 minutes 58.0 seconds,91.38%,"{Put Assessment On-Hold, AR - Satisfied - fina...","[(Send for Assessment, 1604), (Assessment Deci...","[(AR - Reviewed - further information, 9.0 day..."
5,6,781,53 days 4 hours 15 minutes 57 seconds,36.0 days 22.0 hours 52.0 minutes 12.0 seconds,90.14%,"{Put Assessment On-Hold, AR - Satisfied - fina...","[(Assessment Decision - Qualified, 868), (Send...","[(AR - Satisfied - work experience, 8.0 days 7..."
6,7,432,67 days 8 hours 36 minutes 3 seconds,45.0 days 4.0 hours 14.0 minutes 5.5 seconds,80.79%,"{Put Assessment On-Hold, OC - Awaiting Externa...","[(Assessment Decision - Qualified, 510), (Send...","[(Assessment Outcome - Ineligible / Withdrawn,..."
7,8,231,62 days 20 hours 36 minutes 2 seconds,41.0 days 21.0 hours 58.0 minutes 37.0 seconds,87.01%,"{Put Assessment On-Hold, AR - Satisfied - fina...","[(Send for Assessment, 298), (Assessment Decis...","[(Send for Assessment, 9.0 days 14.0 hours 59...."
8,9,158,60 days 2 hours 59 minutes 49 seconds,42.0 days 10.0 hours 59.0 minutes 46.5 seconds,87.34%,"{Put Assessment On-Hold, AR - Satisfied - fina...","[(Assessment Decision - Qualified, 223), (Send...","[(Release Assessment Hold, 8.0 days 4.0 hours ..."
9,10,125,85 days 19 hours 49 minutes 3 seconds,48.0 days 6.0 hours 12.0 minutes 16.0 seconds,86.4%,"{Put Assessment On-Hold, AR - Satisfied - fina...","[(Assessment Decision - Qualified, 187), (Send...",[(Acacdemic - Assessment Decision - No Decisio...


In [13]:
s1_t2.to_csv('engineering_OF_segment1_workflowS2_table.csv', index=False)









